In [1]:
from sentence_transformers import SentenceTransformer, util
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize

import pandas as pd
import sqlite3

C:\Users\keatu\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\keatu\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

In [2]:
# load pretrained BERT model for phrase/sentence similarity
model = SentenceTransformer('all-MiniLM-L6-v2')

In [66]:

# generate map of symptom category (key) and corresponding list of search terms (values)
symptom_map = {
    "blood clot":["clot","blood clot","thrombosis"],
    "heart":["heart problem","heart issue","palpitation","rapid heartbeat","fast heartbeat", "increased heart rate"],
    "stroke":["stroke"],
    "dvt":["deep vein thrombosis"],
    "pulmonary embolism":["embolism","pulmonary embolism"],
    "breathing":["out of breath","shortness of breath","trouble breathing"],
    "lightheaded":["ligthheaded","lightheadedness","faint","dizzy","vertigo"],
    "leg":["leg pain","leg swelling"],
    "skin":["clammy skin","skin discoloration","cyanosis"],
}
# generate list of common self-report symptom phrases
self_report_phrases = ["I had <symptom>","I experienced <symptom>","I felt <symptom>","I suffer from",
"my symptoms included <symptom>","I felt <symptom>","dealing with <symptom>","has anyone else <symptom>"]

# create map between each search term (key) and the resultant self-report phrases and corresponding text embeddings
symptom_embedding_map = {}
for symptom_key in symptom_map:
    symptom_embedding_map[symptom_key] = []
    for symptom in symptom_map[symptom_key]:
        for phrase in self_report_phrases:
            phrase = phrase.replace("<symptom>",symptom)
            embedding = model.encode(phrase, convert_to_tensor=True)
            symptom_embedding_map[symptom_key].append([phrase,embedding])

In [4]:
# read in the data from sqlite DB
datadir = r"C:\Users\keatu\Regis_archive\practicum_data"
dbfile = os.path.join(datadir,"Facebook.db")
con = sqlite3.connect(dbfile)
posts = pd.read_sql("select * from posts",con)
comments = pd.read_sql("select * from comments", con)
replies = pd.read_sql("Select * from replies",con)
con.close()

# grab text/id fields from each data type--treating them all like unique posts
all_text = pd.concat([
                    posts[["user_id","post_id","text"]],
                    comments[["commenter_id","comment_id","comment_text"]].rename(columns={"commenter_id":"user_id","comment_id":"post_id","comment_text":"text"}),
                    replies[["commenter_id","comment_id","comment_text"]].rename(columns={"commenter_id":"user_id","comment_id":"post_id","comment_text":"text"})
                    ], sort = False)

In [72]:
len(all_text)

195682

In [82]:
# compare input sentence embedding to all matching sentence combinations and use threshold cosine similarity score
self_report_df = pd.DataFrame()
threshold = 0.75 # cosine similarity threshold
i=0
for idx, row in all_text.iterrows():
    i=+1
    if (i%10000)==0:
        print("{} completed of {}".format(i,len(all_text)))
    for sent in sent_tokenize(row['text']):
        sent_embed = model.encode(sent, convert_to_tensor=True)
        for term in symptom_embedding_map:
            top_score = 0
            top_match = ""
            for (match_sentence,match_embedding) in symptom_embedding_map[term]:
                cos_score = util.cos_sim(sent_embed, match_embedding).item()
                if cos_score > top_score:
                    top_score = cos_score
                    top_match = match_sentence
            if top_score > threshold:
                self_report_df = self_report_df.append({'user_id':row['user_id'],"post_id":row['post_id'],'sentence':sent,"match_sentence":top_match,"symptom":term,"cos_similarity":top_score}, ignore_index=True)

In [83]:
self_report_df

,cos_similarity,match_sentence,post_id,sentence,symptom,user_id
0,0.759875,I felt leg pain,414608576546564,My legs felt like they were in cement and the ...,leg,100000485450481
1,0.763954,dealing with vertigo,503917610948993,Just wanted to let all know my roommate has en...,lightheaded,100042204009248
2,0.774723,my symptoms included stroke,509523550388399,I am having trouble finding data regarding the...,stroke,1020120541
3,0.760530,I experienced shortness of breath,521006345906786,Shortness of breath only when I do stairs.,breathing,100000275261897
4,0.811827,I experienced shortness of breath,414505476556874,Can't really recall any shortness of breath in...,breathing,100004703479407
...,...,...,...,...,...,...
1439,0.823050,has anyone else shortness of breath,659789512028468,Im short of breath often.,breathing,1074780025
1440,0.835536,I experienced shortness of breath,660736391933780,But I start getting the shortness of breath ag...,breathing,100011018643212
1441,0.788251,my symptoms included shortness of breath,661072865233466,"My main symptoms are neuropathy, and shortness...",breathing,100011018643212
1442,0.765445,my symptoms included vertigo,7467675123257909,"I also have have numbness, vertigo and dizziness.",lightheaded,654418957


In [84]:
self_report_df.to_csv("self_report.csv")